In [1]:
import numpy as np
import pandas as pd

from datetime import datetime

import re

import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei']  # 使用 SimHei 字体
plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题

In [2]:
# 指定文件路径
file_path = "data20240512.xlsx"

# 读取数据集
data = pd.read_excel(file_path, sheet_name='Sheet1')

# 重新命名列
data.rename(columns={'疾病进展发生日期PFS（0代表统计时未进展）': 'PFS', '发生时间OS': 'OS'}, inplace=True)

# 读取可匹配的行数
data = data[:300]
print(data.shape)

(300, 90)


In [3]:
formats = ["%Y%m%d", "%Y/%m/%d", "%Y%m", "%Y.%m.%d", "%Y.%m", "%Y-%m-%d", "%Y-%m"]

def convertDate(text):
    for f in formats:
        try:
            return datetime.strptime(text, f)
        except ValueError:
            pass
    raise ValueError(f"Cannot convert date: {text}")

data['PFS'].fillna(0, inplace=True)
# 创建一个空的DataFrame，并定义列名
PFS_time = pd.DataFrame(columns=['PFS_time'])  
for index, row in data.iterrows():
    line = [row['ICI starting time'].strftime('%Y-%m-%d'), str(row['PFS']).split()[0]]
    try:
        A = convertDate(line[0].strip())
        B = convertDate(line[1].strip()) 
        tdays = (B-A).days
        PFS_time = PFS_time.append({'PFS_time': tdays}, ignore_index=True)
    except ValueError:
        try:
            # print(index, row['PFS'])
            days = int(line[1].strip())
            PFS_time = PFS_time.append({'PFS_time': days}, ignore_index=True)
        except ValueError:
            # print(index, row['PFS']) # 未进展、无随访（5个）
            PFS_time = PFS_time.append({'PFS_time': 0}, ignore_index=True)
            
# 使用concat函数将两个DataFrame按列拼接起来
data = pd.concat([data, PFS_time], axis=1)
 
# Note:数据问题，出现负数，记为0
data['PFS_time'] = data['PFS_time'].apply(lambda x: 0 if x < 0 else x)

# 0记为删失
data['PFS_time'].replace({0: np.nan, np.nan: np.nan})

data['PFS_boolean'] = data['PFS_time'].apply(lambda x: 0 if x == 0 else 1)

In [4]:
data['PFS_time'].value_counts()

0      111
27       3
95       3
62       3
46       3
      ... 
169      1
738      1
231      1
526      1
334      1
Name: PFS_time, Length: 159, dtype: int64

In [5]:
# # 查看数据类型
# for row_index in range(300):
#     # 查看指定行的数据类型
#     value = data.loc[row_index, 'OS']

#     if isinstance(value, datetime):
#         data_type = 'datetime'
#     else:
#         data_type = type(value)

#     print(f"第 {row_index} 行的数据类型为：{data_type}")


# 内容缺失记为：未观测到结局 0
data['OS'].fillna(0, inplace=True)
# 定义一个函数，用于根据数据类型进行转换
def convert_data_type(value):
    if value == 0 or isinstance(value, str):
        return 0
    elif  isinstance(value, datetime) or isinstance(value, int): #   or value.isdigit() 
        # print(value)
        return 1
    else:
        return value  # 其他类型保持不变

# 对列进行转换
data['OS_boolean'] = data['OS'].apply(convert_data_type)
print('数据的结局情况：', data['OS_boolean'].value_counts())


数据的结局情况： 0    230
1     70
Name: OS_boolean, dtype: int64


In [6]:
# 发生OS的时间
# 创建一个空的DataFrame，并定义列名
OS_time = pd.DataFrame(columns=['OS_time'])  
for index, row in data.iterrows():
    line = [row['ICI starting time'].strftime('%Y-%m-%d'), str(row['OS']).split()[0]]
    try:
        A = convertDate(line[0].strip())
        B = convertDate(line[1].strip()) 
        tdays = (B-A).days
        OS_time = OS_time.append({'OS_time': tdays}, ignore_index=True)
    except ValueError:
        # print(index, row['OS'])
        # 0和无数据均认为截止到2024年4月28日生存且没有复发
        A = convertDate(line[0].strip())
        cutoff_date = datetime.strptime('2024-04-28 00:00:00', '%Y-%m-%d %H:%M:%S')
        days = (cutoff_date - A).days 
        OS_time = OS_time.append({'OS_time': days}, ignore_index=True)

        # try:
        #     days = int(line[1].strip()) # 全为0
        #     OS_time = OS_time.append({'OS_time': days}, ignore_index=True)
        # except ValueError: # 带中文
        #     OS_time = OS_time.append({'OS_time': 0}, ignore_index=True)
            
# 使用concat函数将两个DataFrame按列拼接起来
data = pd.concat([data, OS_time], axis=1)


# # 如果A列和B列数据都为0，那么C列数据等于D列数据
# data.loc[(data['PFS_boolean'] == 0) & (data['OS_boolean'] == 0), 'PFS_time'] = data['OS_time']

# 如果A列数据为0，那么C列数据等于D列数据
data.loc[data['PFS_boolean'] == 0, 'PFS_time'] = data['OS_time']

In [7]:
# 将处理后的DataFrame保存为Excel文件
data.to_excel('data_processed_0604.xlsx', index=False)

In [8]:
# 检查A列的数据是否全部大于等于B列
result = (data['OS_time'] >= data['PFS_time']).all()
result

True